In [333]:
import re
import os
import sys
import pandas as pd
import numpy as np

In [334]:
# read datasets

current = pd.read_csv('player_versus_player_details.csv', dtype=str)
missing = pd.read_csv('pokemon_to_add.csv', dtype=str)

current = current.drop(columns='Unnamed: 0')
missing = missing.drop(columns='Unnamed: 0')

current['ID'] = current['ID'].apply(lambda x: x.zfill(4))
missing['ID'] = missing['ID'].apply(lambda x: x.zfill(4))

In [335]:
current.head(1)

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,CHARGE_MOVE_POWER,CHARGED_MOVE_TYPE,CHARGE_MOVE_ENERGY_COST,CHARGE_MOVE_DURATION,DAMAGE WINDOW START
0,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Return,35.0,Normal,33.0,0.7,0.1


In [336]:
missing.head()

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,CHARGE_MOVE
0,0157,Hisuian Typhlosion,Fire,Ghost,Ember,Fire Punch
1,0157,Hisuian Typhlosion,Fire,Ghost,Ember,Overheat
2,0157,Hisuian Typhlosion,Fire,Ghost,Ember,Shadow Ball
3,0157,Hisuian Typhlosion,Fire,Ghost,Ember,Wild Charge
4,0157,Hisuian Typhlosion,Fire,Ghost,Ember,Return


In [337]:
import pandas as pd

# Sample setup for 'current' and 'missing' DataFrames
# Assume these are already loaded with the correct data as per your structure

# Let's ensure there is no issue with data parsing first by printing a few rows
print(current.head())
print(missing.head())

# Function to fill in move details from 'current' if available
def fill_move_details(row, current_df):
    # Check and fetch fast move details
    fast_details = current_df[(current_df['FAST_MOVE'] == row['FAST_MOVE']) & (current_df['FAST_MOVE_POWER'].notna())]
    if not fast_details.empty:
        for col in ['FAST_MOVE_POWER', 'FAST_MOVE_TYPE', 'FAST_ENERGY_BOOST', 'FAST_MOVE_DURATION']:
            row[col] = fast_details.iloc[0][col]

    # Check and fetch charge move details
    charge_details = current_df[(current_df['CHARGE_MOVE'] == row['CHARGE_MOVE']) & (current_df['CHARGE_MOVE_POWER'].notna())]
    if not charge_details.empty:
        for col in ['CHARGE_MOVE_POWER', 'CHARGED_MOVE_TYPE', 'CHARGE_MOVE_ENERGY_COST', 'CHARGE_MOVE_DURATION', 'DAMAGE WINDOW START']:
            row[col] = charge_details.iloc[0][col]

    return row

# Apply the function to each row in 'missing'
missing = missing.apply(fill_move_details, axis=1, current_df=current)

# Print the result to verify
print(missing.head())


     ID       NAME TYPE_ONE TYPE_TWO  FAST_MOVE FAST_MOVE_POWER  \
0  0001  Bulbasaur    Grass   Poison     Tackle             5.0   
1  0001  Bulbasaur    Grass   Poison  Vine Whip             7.0   
2  0001  Bulbasaur    Grass   Poison  Vine Whip             7.0   
3  0001  Bulbasaur    Grass   Poison     Tackle             5.0   
4  0001  Bulbasaur    Grass   Poison     Tackle             5.0   

  FAST_MOVE_TYPE FAST_ENERGY_BOOST FAST_MOVE_DURATION  CHARGE_MOVE  \
0         Normal               5.0                0.5       Return   
1          Grass               6.0                0.6  Sludge Bomb   
2          Grass               6.0                0.6    Seed Bomb   
3         Normal               5.0                0.5  Sludge Bomb   
4         Normal               5.0                0.5    Seed Bomb   

  CHARGE_MOVE_POWER CHARGED_MOVE_TYPE CHARGE_MOVE_ENERGY_COST  \
0              35.0            Normal                    33.0   
1              80.0            Poison         

In [338]:
current.head()# Concatenate the DataFrames
full_df = pd.concat([current, missing], ignore_index=True)

# Drop duplicates based on a sensible subset of columns that should be unique for each entry
full_df.drop_duplicates(subset=['ID', 'NAME', 'FAST_MOVE', 'CHARGE_MOVE'], inplace=True)

# Reorder columns to match the 'current' DataFrame's order
full_df = full_df[current.columns]

# Optionally, sort by 'ID' or any other column if required
full_df.sort_values(by='ID', inplace=True)

In [339]:
full_df[full_df['NAME'] == 'Palkia Origin Forme']

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,CHARGE_MOVE_POWER,CHARGED_MOVE_TYPE,CHARGE_MOVE_ENERGY_COST,CHARGE_MOVE_DURATION,DAMAGE WINDOW START
14093,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Hydro Pump,130.0,Water,100.0,3.3,0.9
14097,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Hydro Pump,130.0,Water,100.0,3.3,0.9
14096,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Fire Blast,140.0,Fire,100.0,4.2,3.1
14095,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Draco Meteor,150.0,Dragon,100.0,3.6,3.0
14090,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Aqua Tail,50.0,Water,33.0,1.9,1.2
14091,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Draco Meteor,150.0,Dragon,100.0,3.6,3.0
14092,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Fire Blast,140.0,Fire,100.0,4.2,3.1
14094,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Aqua Tail,50.0,Water,33.0,1.9,1.2


In [340]:
# Sample data to be added - adjust column names and values as per your actual DataFrame structure
new_data = {
    'ID': ['0484', '0484'],  # Assuming '484' is the Pokedex ID for Palkia
    'NAME': ['Palkia Origin Forme', 'Palkia Origin Forme'],
    'TYPE_ONE': ['Dragon', 'Dragon'],
    'TYPE_TWO': ['Water', 'Water'],  # Assuming Palkia's secondary type is Water
    'FAST_MOVE': ['Dragon Breath', 'Dragon Tail'],
    'FAST_MOVE_POWER': [6.0, 15.0],  # Sample power, adjust based on actual data
    'FAST_MOVE_TYPE': ['Dragon', 'Dragon'],
    'FAST_ENERGY_BOOST': [4.0, 9.0],  # Sample energy, adjust based on actual data
    'FAST_MOVE_DURATION': [0.5, 1.1],  # Sample duration, adjust based on actual data
    'CHARGE_MOVE': ['Spacial Rend', 'Spacial Rend'],
    'CHARGE_MOVE_POWER': [95.0, 95.0],  # As provided
    'CHARGED_MOVE_TYPE': ['Dragon', 'Dragon'],
    'CHARGE_MOVE_ENERGY_COST': [50.0, 50.0],  # As provided
    'CHARGE_MOVE_DURATION': [2.5, 2.5],  # As provided
    'DAMAGE WINDOW START': [1.4, 1.4],  # As provided
}

# Convert dictionary to DataFrame
new_rows = pd.DataFrame(new_data)

In [341]:
# Append the new rows to the full_df
full_df = pd.concat([full_df, new_rows], ignore_index=True)

In [342]:
full_df[full_df['NAME'] == 'Palkia Origin Forme']

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,CHARGE_MOVE_POWER,CHARGED_MOVE_TYPE,CHARGE_MOVE_ENERGY_COST,CHARGE_MOVE_DURATION,DAMAGE WINDOW START
10586,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Hydro Pump,130.0,Water,100.0,3.3,0.9
10592,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Hydro Pump,130.0,Water,100.0,3.3,0.9
10593,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Fire Blast,140.0,Fire,100.0,4.2,3.1
10594,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Draco Meteor,150.0,Dragon,100.0,3.6,3.0
10595,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Aqua Tail,50.0,Water,33.0,1.9,1.2
10596,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Draco Meteor,150.0,Dragon,100.0,3.6,3.0
10597,0484,Palkia Origin Forme,Water,Dragon,Dragon Breath,6.0,Dragon,4.0,0.5,Fire Blast,140.0,Fire,100.0,4.2,3.1
10598,0484,Palkia Origin Forme,Water,Dragon,Dragon Tail,15.0,Dragon,9.0,1.1,Aqua Tail,50.0,Water,33.0,1.9,1.2
15270,0484,Palkia Origin Forme,Dragon,Water,Dragon Breath,6.0,Dragon,4.0,0.5,Spacial Rend,95.0,Dragon,50.0,2.5,1.4
15271,0484,Palkia Origin Forme,Dragon,Water,Dragon Tail,15.0,Dragon,9.0,1.1,Spacial Rend,95.0,Dragon,50.0,2.5,1.4


In [343]:
full_df.shape

(15272, 15)

In [353]:
available_shadows = """
Shadow Bulbasaur
Shadow Ivysaur
Shadow Venusaur
Shadow Charmander
Shadow Charmeleon
Shadow Charizard
Shadow Squirtle
Shadow Wartortle
Shadow Blastoise
Shadow Weedle
Shadow Kakuna
Shadow Beedrill
Shadow Rattata
Shadow Alolan Rattata
Shadow Raticate
Shadow Alolan Raticate
Shadow Ekans
Shadow Arbok
Shadow Sandshrew
Shadow Alolan Sandshrew
Shadow Sandslash
Shadow Alolan Sandslash
Shadow Nidoran
Shadow Nidorina
Shadow Nidoqueen
Shadow Nidoran♂
Shadow Nidorino
Shadow Nidoking
Shadow Vulpix
Shadow Alolan Vulpix
Shadow Ninetales
Shadow Alolan Ninetales
Shadow Zubat
Shadow Golbat
Shadow Oddish
Shadow Gloom
Shadow Vileplume
Shadow Venonat
Shadow Venomoth
Shadow Diglett
Shadow Alolan Diglett
Shadow Dugtrio
Shadow Alolan Dugtrio
Shadow Meowth
Shadow Persian
Shadow Psyduck
Shadow Golduck
Shadow Growlithe
Shadow Arcanine
Shadow Poliwag
Shadow Poliwhirl
Shadow Poliwrath 
Shadow Abra
Shadow Kadabra
Shadow Alakazam
Shadow Machop
Shadow Machoke
Shadow Machamp
Shadow Bellsprout
Shadow Weepinbel
Shadow Victreebel
Shadow Geodude
Shadow Alolan Geodude
Shadow Graveler
Shadow Alolan Graveler
Shadow Golem
Shadow Alolan Golem
Shadow Slowpoke
Shadow Slowbro
Shadow Magnemite
Shadow Magneton
Shadow Grimer
Shadow Alolan Grimer
Shadow Muk
Shadow Alolan Muk
Shadow Shellder
Shadow Cloyster
Shadow Gastly
Shadow Haunter
Shadow Gengar
Shadow Onix
Shadow Drowzee 
Shadow Hypno
Shadow Voltorb
Shadow Electrode
Shadow Exeggcute
Shadow Exeggutor
Shadow Alolan Exeggutor
Shadow Cubone 
Shadow Marowak 
Shadow Alolan Marowak
Shadow Hitmonlee
Shadow Hitmonchan
Shadow Smogon
Shadow Koffing
Shadow Weezing
Shadow Rhyhorn
Shadow Rhydon
Shadow Tangela
Shadow Kangaskhan
Shadow Horsea
Shadow Seadra
Shadow Scyther
Shadow Electabuzz
Shadow Magmar
Shadow Pinsir
Shadow Magikarp
Shadow Gyarados
Shadow Lapras
Shadow Porygon
Shadow Omanyte
Shadow Omastar
Shadow Aerodactyl
Shadow Snorlax
Shadow Articuno
Shadow Zapdos
Shadow Moltres
Shadow Dratini
Shadow Dragonair
Shadow Dragonite
Shadow Mewtwo
Shadow Chikorita
Shadow Bayleef
Shadow Meganium
Shadow Cyndaquil
Shadow Quilava
Shadow Typhlosion
Shadow Totodile
Shadow Croconaw
Shadow Feraligatr
Shadow Ledyba
Shadow Ledian
Shadow Crobat
Shadow Natu
Shadow Xatu
Shadow Mareep
Shadow Flaaffy
Shadow Ampharos
Shadow Bellossom
Shadow Sudowoodo
Shadow Politoed
Shadow Hoppip
Shadow Skiploom
Shadow Jumpluff
Shadow Aipom
Shadow Wooper
Shadow Quagsire
Shadow Murkrow
Shadow Slowking
Shadow Misdreavus
Shadow Wobbuffet
Shadow Girafarig
Shadow Pineco
Shadow Forretress
Shadow Gligar
Shadow Steelix
Shadow Snubbull
Shadow Granbull
Shadow Scizor
Shadow Shuckle
Shadow Sneasel
Shadow Hisuian Sneasel
Shadow Teddiursa
Shadow Ursaring
Shadow Swinub
Shadow Piloswine
Shadow Delibird
Shadow Skarmory
Shadow Houndour
Shadow Houndoom
Shadow Kingdra
Shadow Phanpy
Shadow Donphan
Shadow Porygon
Shadow Stantler
Shadow Raikou
Shadow Entei
Shadow Suicune
Shadow Larvitar
Shadow Pupitar
Shadow Tyranitar
Shadow Lugia
Shadow Ho-Oh
Shadow Sceptile
Shadow Blaziken
Shadow Mudkip
Shadow Marshtomp
Shadow Swampert
Shadow Poochyena
Shadow Mightyena
Shadow Seedot
Shadow Nuzleaf
Shadow Shiftry
Shadow Ralts
Shadow Kirlia
Shadow Gardevoir
Shadow Whismur
Shadow Loudred
Shadow Exploud
Shadow Makuhita
Shadow Hariyama
Shadow Nosepass
Shadow Sableye
Shadow Mawile
Shadow Aron
Shadow Lairon
Shadow Aggron
Shadow Electrike
Shadow Manectric
Shadow Carvanha
Shadow Sharpedo
Shadow Wailmer
Shadow Wailord
Shadow Numel
Shadow Camerupt
Shadow Spoink
Shadow Trapinch
Shadow Vibrava
Shadow Flygon
Shadow Cacnea
Shadow Cacturne
Shadow Whiscash
Shadow Lileep
Shadow Cradily
Shadow Anorith
Shadow Armaldo
Shadow Shuppet
Shadow Banette
Shadow Duskull
Shadow Dusclops
Shadow Absol
Shadow Spheal
Shadow Sealeo
Shadow Walrein
Shadow Bagon
Shadow Shelgon
Shadow Salamence
Shadow Beldum
Shadow Metang
Shadow Metagross
Shadow Regirock
Shadow Regice
Shadow Registeel
Shadow Latias
Shadow Latios
Shadow Turtwig
Shadow Grotle
Shadow Torterra
Shadow Starly
Shadow Staravia
Shadow Staraptor
Shadow Bidoof
Shadow Bibarel
Shadow Shinx
Shadow Luxio
Shadow Luxray
Shadow Cranidos
Shadow Rampardos
Shadow Shieldon
Shadow Bastiodon
Shadow Ambipom
Shadow Drifloon
Shadow Drifblim
Shadow Mismagius
Shadow Honchkrow
Shadow Stunky
Shadow Skuntank
Shadow Garchomp
Shadow Hippopotas
Shadow Hippowdon
Shadow Skorupi
Shadow Drapion
Shadow Snover
Shadow Abomasnow
Shadow Weavile
Shadow Magnezone
Shadow Rhyperior
Shadow Tangrowth
Shadow Electivire
Shadow Magmortar
Shadow Gliscor
Shadow Mamoswine
Shadow Porygon-Z
Shadow Gallade
Shadow Probopass
Shadow Dusknoir
Shadow Regigigas
Shadow Patrat
Shadow Watchog
Shadow Purrloin
Shadow Liepard
Shadow Pidove
Shadow Tranquill
Shadow Unfezant
Shadow Blitzle
Shadow Blitzle
Shadow Zebstrika
Shadow Zebstrika
Shadow Roggenrola
Shadow Boldore
Shadow Gigalith
Shadow Drilbur
Shadow Excadrill
Shadow Sawk
Shadow Ducklett
Shadow Swanna
Shadow Foongus
Shadow Amoonguss
Shadow Litwick
Shadow Lampent
Shadow Chandelure
Shadow Golett
Shadow Golurk
Shadow Ursaluna
Shadow Sneasler
"""

In [344]:
def filter_df_by_name(df, substring):
    filtered_df = df[df['NAME'].str.contains(substring, case=False, na=False)]
    return filtered_df

In [358]:
# Step 1: Count each Pokémon's occurrences in the DataFrame
name_counts = updated_df['NAME'].value_counts()

# Step 2: Filter names that appear only once
unique_pokemon = name_counts[name_counts == 1].index.tolist()

# Output the names of Pokémon that appear only once
print("Pokémon with only one moveset entry in full_df:")
print(unique_pokemon)

Pokémon with only one moveset entry in full_df:
['Smeargle', 'Cosmog', 'Wimpod', 'Cosmoem', 'Ditto', 'Gimmighoul Roaming Form', 'Combee']


In [355]:
def drop_rows_by_exact_name(df, pokemon_name):
    # Filter out rows where the 'NAME' column matches the Pokémon name exactly
    filtered_df = df[df['NAME'] != pokemon_name]
    return filtered_df

In [413]:
updated_df = drop_rows_by_exact_name(updated_df, 'Shadow Palkia Origin Forme')

In [419]:
filter_df_by_name(updated_df,'Cosmoem')

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,CHARGE_MOVE_POWER,CHARGED_MOVE_TYPE,CHARGE_MOVE_ENERGY_COST,CHARGE_MOVE_DURATION,DAMAGE WINDOW START
14339,0790,Cosmoem,Psychic,NaN,Splash,0.0,Water,20.0,1.73,Struggle,35.0,Normal,0.0,2.2,1.2


In [415]:
updated_df = updated_df.sort_values(by='ID', ascending=True)

In [416]:
updated_df

,ID,NAME,TYPE_ONE,TYPE_TWO,FAST_MOVE,FAST_MOVE_POWER,FAST_MOVE_TYPE,FAST_ENERGY_BOOST,FAST_MOVE_DURATION,CHARGE_MOVE,CHARGE_MOVE_POWER,CHARGED_MOVE_TYPE,CHARGE_MOVE_ENERGY_COST,CHARGE_MOVE_DURATION,DAMAGE WINDOW START
0,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Return,35.0,Normal,33.0,0.7,0.1
15,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Power Whip,90.0,Grass,50.0,2.6,1.25
14,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Sludge Bomb,80.0,Poison,50.0,2.3,1.1
13,0001,Bulbasaur,Grass,Poison,Vine Whip,7.0,Grass,6.0,0.6,Seed Bomb,55.0,Grass,33.0,2.1,1.2
12,0001,Bulbasaur,Grass,Poison,Tackle,5.0,Normal,5.0,0.5,Sludge Bomb,80.0,Poison,50.0,2.3,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15265,1000,Gholdengo,Steel,Ghost,Astonish,8.0,Ghost,14.0,1.1,Focus Blast,140.0,Fighting,100.0,3.5,3.0
15266,1000,Gholdengo,Steel,Ghost,Astonish,8.0,Ghost,14.0,1.1,Shadow Ball,100.0,Ghost,50.0,3.0,2.4
15267,1000,Gholdengo,Steel,Ghost,Astonish,8.0,Ghost,14.0,1.1,Dazzling Gleam,100.0,Fairy,50.0,3.5,2.1
15268,1000,Gholdengo,Steel,Ghost,Hex,10.0,Ghost,16.0,1.2,Dazzling Gleam,100.0,Fairy,50.0,3.5,2.1


In [420]:
updated_df.to_csv('final_pokemon_dataset.csv')